In [1]:
from import_src import import_src
import_src()

Append /home/v_user/khiempm/MLOpsVN/src to sys.path


In [8]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import os
import json
from plotly import express as px
from sklearn.preprocessing import LabelEncoder
import mlflow
import time
import threading
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold
import itertools
import pickle

In [9]:
with open("../../data/train_data/phase-3/prob-2/dict_convert.pkl", 'rb') as file_:
    a = pickle.load(file_)

In [10]:
a

{'l2i': {'Denial of Service': 0,
  'Exploits': 1,
  'Information Gathering': 2,
  'Malware': 3,
  'Normal': 4,
  'Other': 5},
 'i2l': {0: 'Denial of Service',
  1: 'Exploits',
  2: 'Information Gathering',
  3: 'Malware',
  4: 'Normal',
  5: 'Other'}}

In [7]:
lb.classes_[0]

'C'

In [3]:
mlflow_evn = "http://localhost:5000"
mlflow.set_tracking_uri(mlflow_evn)
model_uri = os.path.join(
    "models:/", "phase-3_prob-2_lgbm_cv_specific_handle", "5"
)
input_schema = mlflow.models.Model.load(model_uri).get_input_schema().to_dict()
model = mlflow.sklearn.load_model(model_uri)
get_features = [each['name'] for each in input_schema]

In [16]:
link_data_test_raw_32 = '../../data/captured_data/phase-3/prob-2/raw/'
link_data_test_raw_22 = '../../data/captured_data/phase-2/prob-2/raw/'
link_data_test_processed_32 = '../../data/captured_data/phase-3/prob-2/'
train_raw_data_32 = '../../data/raw_data/phase-3/prob-2/raw_train.parquet'
train_raw_data_22 = '../../data/raw_data/phase-2/prob-2/raw_train.parquet'
link_y_data_32 = '../../data/train_data/phase-3/prob-2/train_y.parquet'
link_train_data_32 = '../../data/train_data/phase-3/prob-2/train_x_0.parquet'
link_test_data_32 = '../../data/train_data/phase-3/prob-2/test_x.parquet'
link_y_test_data_32 =   '../../data/train_data/phase-3/prob-2/test_y.parquet'
# link_y_test_data_32 =   '../../data/train_data/phase-3/prob-2/test_y.parquet'


In [17]:
check = pd.read_parquet(link_y_data_32)

In [19]:
check['label']

0           Other
1          Normal
2        Exploits
3          Normal
4           Other
           ...   
77407      Normal
77408    Exploits
77409    Exploits
77410      Normal
77411    Exploits
Name: label, Length: 77412, dtype: object

In [33]:
train_data = pd.read_parquet(link_train_data_32)

In [39]:
train_data[train_data['feature2'] == -1].shape[0] / train_data.shape[0]

0.11394874179713739

In [5]:
def combine_parquet_test(url_test, parquet=False):
    raw_test = pd.DataFrame()
    count_ = 0
    for file_path in os.listdir(url_test):
        if 'parquet' not in file_path or "123.parquet" in file_path:
            continue
        captured_data = pd.read_parquet(f'{url_test}{file_path}')
        if parquet:
            captured_data['parquet_idx'] = count_
        raw_test = pd.concat([raw_test, captured_data])
        count_ += 1
    print(count_)
    return raw_test.reset_index(drop=True)

In [23]:
raw_test_32 = combine_parquet_test(link_data_test_raw_32)
raw_data_32 = pd.read_parquet(train_raw_data_32)
feature_columns = raw_data_32.columns.tolist()
feature_columns.remove("label")
raw_test_32 = raw_test_32[feature_columns]

200


In [44]:
list(itertools.combinations([1,2,3,"K",5], 1))

[(1,), (2,), (3,), ('K',), (5,)]

In [29]:
raw_test_32[~(raw_test_32['feature2'] == 'null') & (raw_test_32['feature3'] == 'unknown') & ~(raw_test_32['feature4'] == 'null')].shape[0] / raw_test_32.shape[0]

0.019080034836393367

In [40]:
t = raw_test_32.groupby('feature4', group_keys=False).apply(lambda x: x.sample(frac=0.05))
t['feature4'].value_counts() / t['feature4'].shape[0]

feature4
INT     0.397459
FIN     0.397103
null    0.125555
CON     0.065132
REQ     0.014395
ECO     0.000267
RST     0.000089
Name: count, dtype: float64

In [17]:
def generate_index_missing_values(data, target_col, per, unavailable_index):
    data['save_index'] = data.index.tolist()
    left_index = list(set(data.index.tolist()) ^ set(unavailable_index))
    left_data = data.loc[left_index].reset_index(drop=True)
    left_data = left_data.groupby(target_col, group_keys=False).apply(lambda x: x.sample(frac=per))
    del unavailable_index, data
    return left_data['save_index'].values.tolist()

def generate_missing_specific_columns(data, list_columns, missing_rate, target_col='label', unavailable_index=[], step=None):
    step = len(list_columns) if step is None else step
    idx_miss = len(missing_rate)
    combinations = list(itertools.combinations(list_columns, step))
    for comb in combinations:
        missing_index = generate_index_missing_values(data[[target_col]], target_col, missing_rate[idx_miss - step], unavailable_index.copy())
        unavailable_index = [*unavailable_index, *missing_index]
        for col in comb:
            data.loc[missing_index, col] = np.nan
            print(f"Col {col} - missing rate {data[col].isna().sum() / data.shape[0]}")
    if step == 1:
        return data 
    return generate_missing_specific_columns(data, list_columns, missing_rate, target_col, unavailable_index, step - 1)


In [24]:
raw_data_32 = generate_missing_specific_columns(raw_data_32, ['feature2', 'feature3', 'feature4'], [0.04, 0.03, 0.02])

/tmp/ipykernel_86183/3653710991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['save_index'] = data.index.tolist()
/tmp/ipykernel_86183/3653710991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['save_index'] = data.index.tolist()
/tmp/ipykernel_86183/3653710991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

Col feature2 - missing rate 0.04001638602013756
Col feature3 - missing rate 0.04001638602013756
Col feature4 - missing rate 0.04001638602013756
Col feature2 - missing rate 0.068799724024924
Col feature3 - missing rate 0.068799724024924
Col feature2 - missing rate 0.09673142019361376
Col feature4 - missing rate 0.06794808218882732
Col feature3 - missing rate 0.09590133891032966
Col feature4 - missing rate 0.09504969707423298
Col feature2 - missing rate 0.11427092990664281
Col feature3 - missing rate 0.11307431922554494
Col feature4 - missing rate 0.11189926909725965


/tmp/ipykernel_86183/3653710991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['save_index'] = data.index.tolist()
/tmp/ipykernel_86183/3653710991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['save_index'] = data.index.tolist()
/tmp/ipykernel_86183/3653710991.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [31]:
raw_data_32[(raw_data_32['feature2'].isna()) & ~(raw_data_32['feature3'].isna()) & ~(raw_data_32['feature4'].isna())].shape[0] / raw_data_32.shape[0]

0.017539509713029044

In [7]:
col_num = feature_columns.copy()
col_num.remove('feature2')
col_num.remove('feature3')
col_num.remove('feature4')

In [9]:
corr_df = raw_data_32[col_num].corr().reset_index(drop=True)

In [7]:
raw_data_32['feature9'].value_counts()

feature9
254.0    60740
31.0     20215
62.0     10113
0.0       1540
64.0        72
1.0         31
29.0        14
63.0        12
60.0        12
255.0        8
32.0         4
30.0         1
Name: count, dtype: int64

In [21]:
raw_test_32['feature10'].value_counts()

feature10
0.0      105910
252.0     69735
29.0      48555
60.0        816
31.0         18
32.0         18
Name: count, dtype: int64

In [17]:
raw_test_32['feature9'].value_counts()

feature9
254.0    148252
31.0      48519
62.0      24194
0.0        3672
64.0        180
63.0         73
32.0         36
29.0         36
255.0        36
1.0          36
60.0         18
Name: count, dtype: int64

In [18]:
raw_data_32['feature9'].value_counts()

feature9
254.0    60740
31.0     20215
62.0     10113
0.0       1540
64.0        72
1.0         31
29.0        14
63.0        12
60.0        12
255.0        8
32.0         4
30.0         1
Name: count, dtype: int64

In [12]:
raw_data_32['feature26'].value_counts()

feature26
57.0      21502
100.0     11319
53.0       3284
73.0       2751
56.0       2666
          ...  
1484.0        1
1397.0        1
769.0         1
930.0         1
1019.0        1
Name: count, Length: 1277, dtype: int64

In [14]:
raw_data_32[raw_data_32['feature40'] == 0]

,feature1,feature2,feature3,feature4,feature5,feature6,feature7,feature8,feature9,feature10,...,feature33,feature34,feature35,feature36,feature37,feature38,feature39,feature40,feature41,label


In [21]:
raw_data_32['feature40'].value_counts()

feature40
1.0     21142
2.0     10253
3.0      8712
4.0      6832
5.0      5693
6.0      4793
7.0      3863
8.0      2721
9.0      2151
10.0     2124
33.0     1644
11.0     1433
12.0     1413
16.0     1264
14.0     1237
17.0     1073
34.0     1057
18.0      919
13.0      910
29.0      830
26.0      827
32.0      765
28.0      757
27.0      728
30.0      703
31.0      699
25.0      648
23.0      642
24.0      616
21.0      560
22.0      547
15.0      513
19.0      480
20.0      462
37.0      457
35.0      445
39.0      366
36.0      348
43.0      330
38.0      292
40.0      275
42.0      192
45.0      180
44.0      165
41.0      159
46.0      119
51.0      118
47.0       65
50.0       61
48.0       50
49.0       41
52.0       20
62.0       20
59.0       14
54.0       13
58.0       12
53.0        9
Name: count, dtype: int64

In [22]:
raw_test_32['feature40'].value_counts()

feature40
1.0     51374
2.0     25150
3.0     21030
4.0     16137
5.0     13668
6.0     11580
7.0      8645
8.0      6475
9.0      5775
10.0     4960
33.0     3980
11.0     3813
12.0     3621
14.0     3151
16.0     2928
17.0     2654
34.0     2465
13.0     2324
26.0     2094
18.0     2016
30.0     1922
25.0     1839
29.0     1837
28.0     1801
22.0     1686
32.0     1679
27.0     1605
31.0     1597
24.0     1534
23.0     1381
15.0     1344
20.0     1333
35.0     1257
21.0     1214
37.0     1108
36.0     1093
19.0      999
43.0      964
39.0      816
38.0      759
40.0      580
41.0      564
42.0      415
45.0      404
44.0      349
51.0      329
46.0      164
47.0      128
50.0      127
48.0      108
62.0       74
49.0       73
53.0       37
52.0       37
58.0       37
54.0       18
Name: count, dtype: int64